<a href="https://colab.research.google.com/github/zhangguanheng66/tutorials/blob/bert_question_answer/fine_tune_BERT_question_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell

rm -r /usr/local/lib/python3.6/dist-packages/torch*
pip install numpy
pip install --pre torch torchvision torchtext -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html

In [ ]:
import torchtext
from torchtext.experimental.transforms import basic_english_normalize
tokenizer = basic_english_normalize()
tokenizer('here is an example')

# Fine-tuning BERT model for Question-Answer with Torchtext

Recently, we release a new torchtext library to address a few issues from OSS community ([torchtext 0.7.0 release note](https://github.com/pytorch/text/releases/tag/v0.7.0-rc3) and 0.8.0 release note). To accelerate research, the new torchtext library will provide reusable, orthogonal, correct, and performant building blocks  (Vectors, Vocab, Tokenizer) for cutting-edge experimentation based on deep knowledge of the NLP research and communities. In addition, we deeply integrate torchtext with broad range of PyTorch capabilities, such as Just-in-Time (JIT), quantization, distributed, and mobile, to enable seamless research-to-production for core end-to-end applications.

In this tutorial, we are going to fune-tune a pretrained BERT model for question-answer task with the new library.


## Step 1: Prepare datasets

We have revisited the very basic components of the torchtext library, including vocab, word vectors, tokenizer backed by regular expression, and sentencepiece. Those are the basic data processing building blocks for raw text string.

### Tokenizer-vocabulary data processing pipeline

In [ ]:
# Prepare data pipeline for question-answer dataset
from torchtext.experimental.transforms import (
    basic_english_normalize,
    TextSequentialTransforms,
    ToLongTensor,
)
from torchtext.experimental.vocab import vocab_from_file
vocab = vocab_from_file(open('bert_vocab.txt', 'r'))
text_pipeline = TextSequentialTransforms(basic_english_normalize(), vocab)
pos_pipeline = ToLongTensor() # work on the position ids of the answer
qa_data_pipeline= {'context': text_pipeline, 'question': text_pipeline,
                   'answers': text_pipeline, 'ans_pos': pos_pipeline}

# Check out raw text
from torchtext.experimental.datasets.raw import SQuAD1
train, dev = SQuAD1()
            
# Use the question-answer dataset template
from torchtext.experimental.datasets.question_answer import QuestionAnswerDataset
train_data = QuestionAnswerDataset([item for item in train], vocab, qa_data_pipeline)
dev_data = QuestionAnswerDataset([item for item in dev], vocab, qa_data_pipeline)

### Word-vector embedding data processing pipeline

Word embeddings are a type of word representation that allows words with similar meaning to have a similar representation. FastText and GloVe are well established baseline word vectors in the NLP community. In the new torchtext library, a Vector object supports the mapping between tokens and their corresponding vector representation (i.e. word embeddings).

In [ ]:
from torchtext.experimental.transforms import (
    basic_english_normalize,
    TextSequentialTransforms,
)
from torchtext.experimental.vectors import FastText
vector = FastText()
word_vector_pipeline = TextSequentialTransforms(basic_english_normalize(), vector)

### SentencePiece data processing pipeline

SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training. For sentencepiece transforms in torchtext, both subword units (e.g., byte-pair-encoding (BPE) ) and unigram language model are supported.

In [ ]:
from torchtext.experimental.transforms import (
    pretrained_spm,
    SentencePieceTransform,
)
spm = pretrained_spm()
spm_transform = SentencePieceTransform(spm)

### JIT support for the data processing pipeline

The new building blocks in torchtext library is compatible with torch.jit.script. TorchScript is a way to create serializable and optimizable models from PyTorch code. Any TorchScript program can be saved from a Python process and loaded in a process where there is no Python dependency. The data processing pipelines above can be converted and run on the JIT mode without Python dependency


In [ ]:
import torch
jit_text_pipeline = torch.jit.script(text_pipeline.to_ivalue())
jit_pos_pipeline = torch.jit.script(pos_pipeline)

## Step 2: Data Iterator

The PyTorch data loading utility is the [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) class. It works with a map-style dataset that implements the __getitem__() and __len__() protocols, and represents a map from indices/keys to data sampl*e*s. Before sending to the model, collate_fn function works on a batch of samples generated from DataLoader. 

In [ ]:
from torch.utils.data import DataLoader
BATCH_SIZE = 16

# [TODO]
# Need to integrate with torchtext.experimental.transforms.PadTransform
def collate_batch(batch):
    seq_list, ans_pos_list, tok_type = [], [], []
    for item in batch:
        qa_item = torch.cat((torch.tensor([cls_id]), item['question'], torch.tensor([sep_id]),
                             item['context'], torch.tensor([sep_id])))
        seq_list.append(qa_item)
        pos_list = [pos + item['question'].size(0) + 2 for pos in item['ans_pos']]
        ans_pos_list.append(pos_list)
        tok_type.append(torch.cat((torch.zeros((item['question'].size(0) + 2)),
                                   torch.ones((args.bptt - item['question'].size(0) - 2)))))
    _ans_pos_list = []
    for pos in zip(*ans_pos_list):
        _ans_pos_list.append(torch.stack(list(pos)))
    return torch.stack(seq_list).long().t().contiguous().to(device), \
        _ans_pos_list, torch.stack(tok_type).long().t().contiguous().to(device)
        
dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch)

## Step 3: Model for Question-Answer Task

A BERT model was pretrained with the masked language modeling task and next-sentence task according to the paper - [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805).  torchtext.datasets.WikiText103 and BookCorpus were used to pre-trained the model.

Here is the layout of the model for the question-answer task. On the top of the BERT model, there is a linear layer to project the position of the answer.

In [ ]:
import torch
class QuestionAnswerTask(torch.nn.Module):
    """Contain a pretrain BERT model and a linear layer."""

    def __init__(self, bert_model):
        super(QuestionAnswerTask, self).__init__()
        self.bert_model = bert_model
        self.activation = nn.functional.gelu
        self.qa_span = nn.Linear(bert_model.ninp, 2)

    def forward(self, src, token_type_input):
        output = self.bert_model(src, token_type_input)
        # transpose output (S, N, E) to (N, S, E)
        output = output.transpose(0, 1)
        output = self.activation(output)
        pos_output = self.qa_span(output)
        start_pos, end_pos = pos_output.split(1, dim=-1)
        start_pos = start_pos.squeeze(-1)
        end_pos = end_pos.squeeze(-1)
        return start_pos, end_pos

 The pretrained model is available here

In [ ]:
%%shell
wget https://pytorch.s3.amazonaws.com/models/text/torchtext_bert_example/full_ns_bert.pt

and can be loaded to the question answer task

In [ ]:
pretrained_bert = torch.load('full_ns_bert.pt')
qa_model = QuestionAnswerTask(pretrained_bert)

## Step 4: Fine-tuning the Model

Then, we fine-tune the BERT model with the question-answer task based on the SQuAD1 dataset

In [ ]:
def fine_tune(model, optimizer, criterion, batch_size):
    model.train()
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True,
                            collate_fn=collate_batch)
    for idx, (seq_input, ans_pos, tok_type) in enumerate(dataloader):
        optimizer.zero_grad()
        start_pos, end_pos = model(seq_input, token_type_input=tok_type)
        target_start_pos, target_end_pos = ans_pos[0].to(device).split(1, dim=-1)
        target_start_pos = target_start_pos.squeeze(-1)
        target_end_pos = target_end_pos.squeeze(-1)
        loss = (criterion(start_pos, target_start_pos) + criterion(end_pos, target_end_pos)) / 2
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_loss += loss.item()
        if idx % log_interval == 0 and idx > 0:
            cur_loss = total_loss / log_interval
            train_loss_log[-1] = cur_loss
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:05.5f} | '
                  'ms/batch {:5.2f} | '
                  'loss {:5.2f} | ppl {:8.2f}'.format(epoch, idx,
                                                      len(train_dataset) // batch_size,
                                                      scheduler.get_last_lr()[0],
                                                      elapsed * 1000 / log_interval,
                                                      cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
            
# Hyperparameters
EPOCHS = 10 # epoch
LR = 0.5  # learning rate
BATCH_SIZE = 72 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(qa_model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    fine_tune(qa_model, optimizer, criterion, BATCH_SIZE)